In [ ]:
from pathlib import Path

PROJECT_PATH = Path().resolve().parent
SUPPLEMENT_PATH = PROJECT_PATH / 'supplement'

In [ ]:
def add_line(file, line):
    file.write(f'{line}\n')
    
def start_doc(file, bib_filename, title):
    packages = [
        '\\usepackage[T1]{fontenc}',
        '\\usepackage[utf8]{inputenc}',
        '\\usepackage{lmodern}',
        '\\usepackage{textcomp}',
        '\\usepackage{lastpage}',
        '\\usepackage{graphicx}',
        '\\usepackage{longtable}',
        '\\usepackage[sorting=none]{biblatex}',
        '\\usepackage{booktabs}',
    ]
    add_line(app_file, '\\documentclass{article}')
    for line in packages:
        add_line(app_file, line)
    add_line(app_file, f'\\addbibresource{{{bib_filename}.bib}}')
    add_line(app_file, f'\\title{{{title}}}')
    add_line(app_file, '\\begin{document}')
    
def finish_doc(file):
    add_line(app_file, '\\printbibliography')
    add_line(app_file, '\\end{document}')
    
def populate_doc(file, elements):
    for element in elements:
        file.write(element)
    file.write('\n')

In [ ]:
with open(SUPPLEMENT_PATH / 'appendix.tex', 'w') as app_file:
    start_doc(app_file, 'austcovid', 'Supplemental Appendix')
    populate_doc(app_file, ['We built a model. ', 'It was enjoyable. '])
    finish_doc(app_file)